In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
train = pd.read_csv(os.path.join(path,"merged_train.csv"))
test = pd.read_csv(os.path.join(path,"merged_test.csv"))

In [3]:
reg = train.groupby("지역").mean()["등록차량수"].reset_index().rename(columns={'등록차량수':'지역_등록차량수'})
train= pd.merge(train, reg, on="지역", how="left")
train["지역x세대당인구"] =  train["지역_등록차량수"]*train['세대당_인구']
test= pd.merge(test, reg, on="지역", how="left")
test["지역x세대당인구"]= test["지역_등록차량수"]*test["세대당_인구"]

*상가 데이터는 제외*

In [4]:
train = train[train['임대건물구분']=='아파트']
test = test[test['임대건물구분']=='아파트']

### 전용면적 단위 통일
ex. `전용면적` : 20-30인 경우 20으로 통일

In [5]:
train['전용면적'] = train['전용면적']//10 * 10
train['전용면적'] = train['전용면적'].astype('int').astype('str')

In [6]:
test['전용면적'] = test['전용면적']//10 * 10
test['전용면적'] = test['전용면적'].astype('int').astype('str')

In [7]:
not_unique = ['자격유형','임대보증금','임대료','전용면적별세대수','전용면적','공급유형']

### 데이터 재구성

- `단지코드`, `전용면적`, `공급유형` 기준으로 데이터 일차원으로 병합

*면적 별로 (20,30) 임대료, 임대보증금의 차이가 커서 단순 평균내는 것은 정보손실 우려가 있음*

- `전용면적별세대수_비율` : 아파트 총세대수 중에서 해당 면적이 차지하고 있는 세대수 비율


- `단지내주차면수_비율` , `등록차량수_비율` : `전용면적별세대수_비율`을 곱해서 새롭게 변수 생성 


- `임대료` `임대보증금` : 평균값 사용


- `인구` : `전용면적별세대수_비율`을 곱해서 그 전용면적에 속한 아파트의 인구분포를 살펴보고자 함

In [8]:
tmp = train.groupby(['단지코드','전용면적','공급유형'],as_index=False).agg({'전용면적별세대수':'sum','임대료':'mean','임대보증금':'mean'})
train = pd.merge(train.drop(not_unique,axis=1).drop_duplicates(),tmp)

train['전용면적별세대수_비율'] = train['전용면적별세대수']/train['총세대수'] 
train['면적별_등록차량수'] = train['등록차량수'] * train['전용면적별세대수_비율']
train['면적별_단지내주차면수'] = train['단지내주차면수'] * train['전용면적별세대수_비율']

In [9]:
tmp = test.groupby(['단지코드','전용면적','공급유형'],as_index=False).agg({'전용면적별세대수':'sum','임대료':'mean','임대보증금':'mean'})
test = pd.merge(test.drop(not_unique,axis=1).drop_duplicates(),tmp)

test['전용면적별세대수_비율'] = test['전용면적별세대수']/test['총세대수'] 
test['면적별_단지내주차면수'] = test['단지내주차면수'] * test['전용면적별세대수_비율']

In [10]:
age_ratio = ['0~19세_비율', '20~39세_비율','40~69세_비율', '70세이상_비율']
age_col = []
for col in age_ratio:
    col_ = col.split('_')[0] + '_인구수'
    age_col.append(col_)
    train[col_] = train[col] * train['전용면적별세대수']
    test[col_] = test[col] * test['전용면적별세대수']

In [11]:
train.drop(age_ratio,axis=1,inplace=True)
test.drop(age_ratio,axis=1,inplace=True)

### 파생변수 생성

`공가비율` `지역별_등록차량수`

In [12]:
train['공가비율'] = train['공가수'] / train['총세대수']
test['공가비율'] = test['공가수'] / test['총세대수']

In [13]:
tmp = train.groupby(['지역','전용면적'])[['면적별_등록차량수']].mean().reset_index().rename(columns={'면적별_등록차량수':'지역별_등록차량수'})
train = pd.merge(train,tmp)
test = pd.merge(test,tmp)
tmp.head(3)

,지역,전용면적,지역별_등록차량수
0,강원도,10,112.125000
1,강원도,20,105.336175
2,강원도,30,140.168372


### 이상치제거

In [14]:
train = train[train['단지코드']!='C1722']
train = train.reset_index(drop=True)

### 카테고리변수 처리

`공급유형` `전용면적`

In [15]:
train = pd.concat([train,pd.get_dummies(train['공급유형'])],axis=1)
train = pd.concat([train,pd.get_dummies(train['전용면적'],prefix='면적')],axis=1)

In [16]:
test = pd.concat([test,pd.get_dummies(test['공급유형'])],axis=1)
test = pd.concat([test,pd.get_dummies(test['전용면적'],prefix='면적')],axis=1)

In [17]:
for col in ['공공분양', '공공임대(5년)', '장기전세', '면적_60']:
    test[col] = 0

# 모델링

In [18]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [19]:
col = ['면적별_단지내주차면수','세대당_인구','공가비율','총세대수','지역별_등록차량수','임대료','임대보증금']
col = col + age_col
col = col + ['공공분양', '공공임대(10년)',
       '공공임대(50년)', '공공임대(5년)', '공공임대(분납)', '국민임대', '영구임대', '장기전세', '행복주택']
col = col + ['면적_10', '면적_20', '면적_30', '면적_40', '면적_50', '면적_60', '면적_70', '면적_80']

In [20]:
train_X = train[col]
train_y = train['면적별_등록차량수']
test_X = test[col]
train_X.head(2)

,면적별_단지내주차면수,세대당_인구,공가비율,총세대수,지역별_등록차량수,임대료,임대보증금,0~19세_인구수,20~39세_인구수,40~69세_인구수,70세이상_인구수,공공분양,공공임대(10년),공공임대(50년),공공임대(5년),공공임대(분납),국민임대,영구임대,장기전세,행복주택,면적_10,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80
0,407.60367,2.3,0.031193,545,184.083403,99066.666667,11520000.0,69.638961,65.864318,185.885165,34.611555,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,52.80000,2.6,0.000000,560,184.083403,80800.000000,3631000.0,41.704849,48.146343,90.861857,11.286950,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [21]:
lasso = Lasso()
lasso.fit(train_X,train_y)
test['pred'] = lasso.predict(test_X)

### 제출

In [22]:
submit = test.groupby(['단지코드'])[['pred']].sum().reset_index()
submit.columns = ['code','num']
submit.loc[147,:] = ['C1327',0]
submit.loc[148,:] = ['C2335',0]
submit.loc[149,:] = ['C2675',0]

In [23]:
submit.to_csv("submit1201.csv",index=False)